In [1]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import math

In [79]:
def noise_gen(m,n):

    data = pd.read_csv("logReg.csv")  
    X=data.iloc[:,:-1].values
    Lambda=1
    epsilon=100 # diff private loss parameter
    scale = 2/( len(X) * Lambda * epsilon)
    
    t=[]
    noiseS = [[t for i in range(n)] for j in range(n)]
    nv=[]
    noise = [[nv for i in range(n)] for j in range(n)]
    for i in range(n):
    
        #generate random values for S trick
        
        for j in range(n):          
            noiseS[i][j]=random.randint(1,10)
            
        from ipynb.fs.full.ShamirSecretSharing import shamir_split            
        #generate DP noise vector (Gamma - Gamma) 
        
        for j in range(n):
            a=np.random.gamma(1/n,scale, 1)
            b=np.random.gamma(1/n,scale, 1)
            noise[i][j]= b-a
            val=noise[i][j]+noiseS[i][j]
            pickle.dump( val, open( "noise"+str(i)+str(j)+".p", "wb" ) )
            
      #  for i in range(n):
        for j in range(n):
                l = shamir_split(m,n,noiseS[i][j])
                for k in range(n):
                     pickle.dump( l[k][1], open( "t"+str(i)+str(j)+str(k)+".p", "wb" ) ) 
                     
    #for i in range(n):
        for k in range(n):
                sum=0
                for j in range(n):
                        sum-=pickle.load( open( "t"+str(i)+str(j)+str(k)+".p", "rb" ) )
                pickle.dump( sum, open( "s"+str(i)+str(k)+".p", "wb" ) ) 
    
    return noiseS

def calculate(m, n, weight_i): # n - number of parties/clients , weight_i - ith weight
    
    data = pd.read_csv("logReg.csv")  
    X=data.iloc[:,:-1].values

    
    nv1=[]
    noiseR = [[nv1 for i in range(n)] for j in range(n)]
    
    foo=[]
    r = [[foo for i in range(n)] for j in range(n)]

    noiseR=noise_gen(m,n) # all parties generate necesar
    
    # Parties calculate shares and DP noise and send it to others
    for i in range(n):
        x=weight_i 
        
        w=[]      
                          
        for j in range(n): 
            
            if(i==j):
                for k in range(n):
                    x+=pickle.load( open( "noise"+str(k)+str(i)+".p", "rb" ) )

        from ipynb.fs.full.ShamirSecretSharing import shamir_split
        init_weight=shamir_split(m,n,x.round()) # round here 
        w=np.arange(n)     
  

        for j in range(n):          
            r[i][j]=random.randint(1,1000)
                
        for j in range(n):
            w[j]=init_weight[j][1]
            pickle.dump( w[j]+r[i][j], open( "weight"+str(i)+str(j)+".p", "wb" ) ) 
 
    

    
#----------------------------------------------------------------------------------------------------------------#

    from ipynb.fs.full.ShamirSecretSharing import shamir_split
    #Parties calculate their share to be sent to server    
    
    for i in range(n):
        
        sum=0 
                
        for j in range(n): 
                       
            if(i==j):
                for k in range(n):
                    sum+=pickle.load( open( "weight"+str(k)+str(i)+".p", "rb" ) )
                sum+=pickle.load( open( "s"+str(i)+str(j)+".p", "rb" ) ) # s is p in the algorithm
                sum-=r[i][j]
                pickle.dump( sum, open( "ro"+str(i)+str(j)+".p", "wb" ) )
            else: 
                p=pickle.load( open( "s"+str(i)+str(j)+".p", "rb" ) )
                pickle.dump( p -r[i][j] , open( "ro"+str(i)+str(j)+".p", "wb" ) ) 
        
#----------------------------------------------------------------------------------------------------------------#    
    #Server calculates weighted mean
    
    coeff=[]
    for i in range(n):
        sum=0 
        for j in range(n):
             sum += pickle.load( open( "ro"+str(j)+str(i)+".p", "rb" ) ) 
       # print(sum)
        coeff.append((i,sum))
    
    
    from ipynb.fs.full.ShamirSecretSharing import shamir_resolve
    print(shamir_resolve(res)  )      

    

In [3]:
from ipynb.fs.full.LogisticRegression import logisticRegression
x=logisticRegression()[1]
x

array([1.44618453])

In [80]:
calculate(2,3,5)
from ipynb.fs.full.ShamirSecretSharing import shamir_resolve


15
